# Process data

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Retrieve raw data

In [2]:
from google.cloud import storage
client = storage.Client()

bucket_name = "tdt4173-datasets"
bucket = client.get_bucket(bucket_name)

blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

cats-vs-dogs/
cats-vs-dogs/processed/catsvsdogs.npy
cats-vs-dogs/raw-data/
cats-vs-dogs/raw-data/catsanddogs.zip


In [ ]:
DATASET = "faces/videos/faces.zip"
blob = bucket.get_blob(DATASET)
blob.download_to_filename("data/faces.zip")

# faces.zip should have the following structure: class/video.mp4

In [ ]:
mkdir data/faces
!unzip data/faces.zip data/faces

# Extract images

In [ ]:
# TODO: Just resize image and keep aspect ratio
def preprocess_image(img, img_size=128):
    # Convert to grayscale
    # Note: we convert from BGR as VideoCapture 
    # converts the images to BGR color frame by defualt
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Make image square by filling with black padding
    h, w = img.shape
    img = cv2.copyMakeBorder(
        img,
        top=0,
        right=max(0, h-w),
        bottom=max(0, w-h),
        left=0,
        borderType=0,
    )

    # Resize image to specified size
    img = cv2.resize(img, (img_size, img_size))
    return img

In [ ]:
def max_label(name, folder):
    # Find maximum label of files named as "name_##.jpg" in directory
    highest = 0
    for file in os.listdir(folder):
        if name not in file:
            continue
        label = file.rpartition(os.sep)[2].split(".")[0].rpartition("_")[2]
        highest = max(highest, int(label))
    return highest

In [ ]:
def extract_images_from_video(video, target_dir=None, file_prefix=None, img_size=128, processing_func=preprocess_images, silent=False):    
    """
    Saves every single frame of a video as images.
    Converts to greyscale, pads with black to make square images and resizes.
    """
    
    vidcap = cv2.VideoCapture(video)
    
    video_name = video.rpartition(os.sep)[2].split(".")[0]
    
    # Where to save images
    if not target_dir:
        target_dir = os.getcwd() + f"/images_from_video_{video_name}"
        os.makedirs(target_dir, exist_ok=True) 

    # Use video file name as prefix if not specified
    if not file_prefix:
        file_prefix = video_name

    label = max_label(file_prefix, target_dir)
    count = 0
    
    while True:
        # Read image from video
        success, image = vidcap.read()
        
        # if frame is read correctly, succes is True
        if not success:
            print("Can't receive frame (stream end?). Exiting ...")
            return count
        
        # Convert to greyscale, make square and resize
        image = processing_func(image, img_size)

        # Save to file
        label += 1
        count += 1
        file_name = f"{file_prefix}_{str(count)}.jpg"
        path = os.path.join(target_dir, file_name)
        cv2.imwrite(path, image)

        # Check that image is not corrupted
        if cv2.imread(path) is None:
            print(f"WARNING: image corrupted at path {path}")
            os.remove(path)
        else:
            if not silent:
                print(f'Image successfully written at {path}')


In [ ]:
KJARTAN = "data/faces/Kjartan"
LARS = "data/faces/Lars"
MORGAN = "data/faces/Morgan"
OTHER = "data/faces/Other"

LABELS = {KJARTAN: 0, LARS: 1, MORGAN: 2, OTHER: 3}
counts = {KJARTAN: 0, LARS: 0, MORGAN: 0, OTHER: 0}


# For each class, extract all frames from all videos, preprocess image and save to data/faces/label
for label in LABELS:
    count = 0
    for video in tqdm(os.listdir(label)):
        # Extract each frame of video, preprocess and save to directory
        target_dir = f"data/faces/images/{label.rpartition("/")[2]}"
        count += extract_images_from_video(video, target_dir, processing_func=preprocess_image)
    counts[label] = count
    

print(f"Kjartan: {counts[KJARTAN]}, Lars: {counts[LARS]}, Morgan: {counts[MORGAN]}, Other: {counts[Other]")


# Save images to Cloud Storage

In [ ]:
!zip -r data/faces/images.zip data/faces/images

In [ ]:
cloud_directory = "faces/images/images.zip"
blob = bucket.blob(cloud_directory)

source_file_name = "data/faces/images.zip"
blob.upload_from_filename(source_file_name)

# Notes for data collection

- Currently every single frame of the video is used, so each frame needs to contain the face of the person
- All videos should use the same camera settings (resolution etc)
- Should film with as low resolution as possible
- Low framerate is probably ideal
- When saving videos, store them as `folder/Class/video.mp4` and zip `folder`